In [1]:
#用于ensemble
import collections
import os
import random
import tarfile
import torch
from torch import nn
import torchtext.vocab as Vocab
import torch.utils.data as Data
import pickle as pk
import copy
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import torch.nn.functional as F
from IPython.display import display,HTML
import os
from torch.nn.utils.rnn import pad_sequence
from torch.nn.utils.rnn import pack_padded_sequence
from torch.nn.utils.rnn import pad_packed_sequence
from torch.nn.utils.rnn import pack_sequence
from tqdm import tqdm_notebook as tqdm
import math
from transformers import AdamW
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import pandas as pd
import lightgbm as lgb
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import time
import itertools
import multiprocessing
from transformers import *
import json
device=torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
np.set_printoptions(suppress=True, threshold=np.nan)
pd.set_option('float_format', lambda x: '%.3f' % x)

In [2]:
do_lower_case=True
max_len=256
device=torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
# bert_dir="./bert-pytorch-chinese/"
# vocab="bert-base-chinese-vocab.txt"
# config_file="bert_config.json"
bert_dir="./roberta-zh-wwm-pytorch/"
vocab="vocab.txt"
config_file="bert_config.json"
tokenizer=BertTokenizer.from_pretrained(os.path.join(bert_dir,vocab),do_lower_case=do_lower_case)
def index_token(text_raw):
    sub_text = []
    buff = ""
    for char in text_raw:
        if is_chinese_char(ord(char)) or is_punctuation(char):
            if buff != "":
                sub_text.append(buff)
                buff = ""
            sub_text.append(char)
        else:
            buff += char
    if buff != "":
        sub_text.append(buff)
    tok_to_orig_start_index = []
    tok_to_orig_end_index = []
    orig_to_tok_index = []
    tokens = []
    text_tmp = ''
    for (i, token) in enumerate(sub_text):
        orig_to_tok_index.append(len(tokens))
        sub_tokens = tokenizer.tokenize(token)
        text_tmp += token
        for sub_token in sub_tokens:
            tok_to_orig_start_index.append(len(text_tmp) - len(token))
            tok_to_orig_end_index.append(len(text_tmp) - 1)
            tokens.append(sub_token)
        else:
            continue
        break
    return tok_to_orig_start_index,tok_to_orig_end_index,orig_to_tok_index
import unicodedata
def _is_whitespace(char):
    """Checks whether `chars` is a whitespace character."""
    # \t, \n, and \r are technically contorl characters but we treat them
    # as whitespace since they are generally considered as such.
    if char == " " or char == "\t" or char == "\n" or char == "\r":
        return True
    cat = unicodedata.category(char)
    if cat == "Zs":
        return True
    return False
def is_chinese_char(cp):
    """Checks whether CP is the codepoint of a CJK character."""
    # This defines a "chinese character" as anything in the CJK Unicode block:
    #   https://en.wikipedia.org/wiki/CJK_Unified_Ideographs_(Unicode_block)
    #
    # Note that the CJK Unicode block is NOT all Japanese and Korean characters,
    # despite its name. The modern Korean Hangul alphabet is a different block,
    # as is Japanese Hiragana and Katakana. Those alphabets are used to write
    # space-separated words, so they are not treated specially and handled
    # like the all of the other languages.
    if (
        (cp >= 0x4E00 and cp <= 0x9FFF)
        or (cp >= 0x3400 and cp <= 0x4DBF)  #
        or (cp >= 0x20000 and cp <= 0x2A6DF)  #
        or (cp >= 0x2A700 and cp <= 0x2B73F)  #
        or (cp >= 0x2B740 and cp <= 0x2B81F)  #
        or (cp >= 0x2B820 and cp <= 0x2CEAF)  #
        or (cp >= 0xF900 and cp <= 0xFAFF)
        or (cp >= 0x2F800 and cp <= 0x2FA1F)  #
    ):  #
        return True

    return False
def is_punctuation(char):
    """Checks whether `chars` is a punctuation character."""
    cp = ord(char)
    # We treat all non-letter/number ASCII as punctuation.
    # Characters such as "^", "$", and "`" are not in the Unicode
    # Punctuation class but we treat them as punctuation anyways, for
    # consistency.
    if (cp >= 33 and cp <= 47) or (cp >= 58 and cp <= 64) or (cp >= 91 and cp <= 96) or (cp >= 123 and cp <= 126):
        return True
    cat = unicodedata.category(char)
    if cat.startswith("P"):
        return True
    return False

Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated


In [3]:
text_data=[]
with open("./dataset/train_data/new_train_data.json","r") as r:
    raw_data=r.readlines()
    for d in raw_data:
        text_data.append(json.loads(d))
test_text_data=[]
with open("./dataset/test1_data/new_test1_data.json","r") as r:
    raw_data=r.readlines()
    for d in raw_data:
        test_text_data.append(json.loads(d))
test2_text_data=[]
with open("./dataset/test2_data/new_test2_data.json","r") as r:
    raw_data=r.readlines()
    for d in raw_data:
        test2_text_data.append(json.loads(d))
schema=[]
with open("./dataset/schema.json","r") as r:
    raw_schema=r.readlines()
    for d in raw_schema:
        schema.append(json.loads(d))
rels=set()
special_rels=set()
for e in schema:
        if len(e['object_type'].keys())==1:
            rels.add(e["predicate"])
        else:
            special_rels.add(e["predicate"])
            for key in e['object_type'].keys():
                rels.add(e['predicate']+"_"+key)
if not os.path.exists("./dataset/dict.pk"):
    special_rels=list(special_rels)
    id2rels=list(rels)
    rels2id=dict([(rel,idx) for idx,rel in enumerate(id2rels)])
    id2labels=["O","B-OBJ","I-OBJ","B-SUB","I-SUB","[category]","[SEP]","[CLS]","[PAD]"]
    label2ids=dict([ (label,idx) for idx,label in enumerate(id2labels)])
    pk.dump([special_rels,id2rels,rels2id,id2labels,label2ids],open("./dataset/dict.pk","wb"))
else:
    print("loading dict...")
    special_rels,id2rels,rels2id,id2labels,label2ids=pk.load(open("./dataset/dict.pk","rb"))
id2kglabels=['O','I']+['B-'+e+"-SUB" for e in id2rels]+['B-'+e+"-OB" for e in id2rels]
kglabels2id=dict([ (label,idx) for idx,label in enumerate(id2kglabels)])
new_id2labels=["O","B-OBJ","E-OBJ","B-SUB","E-SUB","[category]","[SEP]","[CLS]","[PAD]"]
new_label2ids=dict([ (label,idx) for idx,label in enumerate(new_id2labels)])
new_id2kglabels=['O']+['B-'+e+"-SUB" for e in id2rels]+['B-'+e+"-OB" for e in id2rels]+['E-'+e+"-SUB" for e in id2rels]+['E-'+e+"-OB" for e in id2rels]
new_kglabels2id=dict([ (label,idx) for idx,label in enumerate(new_id2kglabels)])
id2reltype=[[] for i in range(len(id2rels))]
for e in schema:
    if len(e['object_type'].keys())==1:
        rel=e["predicate"]
        ids=rels2id[rel]
        id2reltype[ids].append(e)
    else:
        for key in e['object_type'].keys():
            rel=e['predicate']+"_"+key
            ids=rels2id[rel]
            temp_e=copy.deepcopy(e)
            poped_keys=[]
            for k in temp_e['object_type'].keys():
                if k!=key:
                    poped_keys.append(k)
            for k in poped_keys:
                 temp_e['object_type'].pop(k)
            id2reltype[ids].append(temp_e)
id2schema=[e[0] for e in id2reltype]
id2rel_text=[[] for i in range(len(id2rels))]
id2rel_rawtext=[[] for i in range(len(id2rels))]
id2rel_token2text=[[] for i in range(len(id2rels))]
for rel in range(len(id2rels)):
    if id2rels[rel].split("_")[0] not in special_rels:
        cls_text=id2schema[rel]['subject_type']+","+id2schema[rel]['predicate']+","+id2schema[rel]['object_type']['@value']
    else:
        cls_text=id2schema[rel]['subject_type']+","+id2schema[rel]['predicate']+","+id2schema[rel]['object_type'][id2rels[rel].split("_")[1]]
    id2rel_text[rel]=tokenizer.tokenize(cls_text)
    id2rel_rawtext[rel]=cls_text
    id2rel_token2text[rel]=index_token(cls_text)[0]
    assert len(id2rel_token2text[rel])==len(id2rel_text[rel])
if not os.path.exists("./middle_data/rel_data_postag.pk"):
    jieba.enable_paddle() 
    jieba.enable_parallel(8)
    rel_cut_words=[]
    rel_cut_tags=[]
    for idx in tqdm(range(len(id2rel_rawtext))):
        words = pseg.lcut(id2rel_rawtext[idx],use_paddle=True) #jieba默认模式
        new_words=[w for w,t in words]
        new_tags=[t for w,t in words]
        rel_cut_words.append([idx,new_words])
        rel_cut_tags.append([idx,new_tags])
    rel_cut_words=[e[1] for e in sorted(rel_cut_words,key=lambda x:x[0])]
    rel_cut_tags=[e[1] for e in sorted(rel_cut_tags,key=lambda x:x[0])]
    rel_data_postag=[]
    for idx in tqdm(range(len(id2rel_rawtext))):
        assert len(id2rel_rawtext[idx].strip())==len("".join(rel_cut_words[idx]))
        indexs=[]
        cur_length=0
        for e in rel_cut_words[idx]:
            indexs.append([cur_length,cur_length+len(e)-1])
            cur_length+=len(e)
        pos_label=np.zeros(len(id2rel_rawtext[idx])).astype(np.int8)
        for i,(b,e) in enumerate(indexs):
            assert (id2rel_rawtext[idx][b]==rel_cut_words[idx][i][0] or _is_whitespace(id2rel_rawtext[idx].strip()[b])\
                   or _is_whitespace(rel_cut_words[idx][i][0])) \
                    and (id2rel_rawtext[idx].strip()[e]==rel_cut_words[idx][i][-1] \
                         or _is_whitespace(id2rel_rawtext[idx].strip()[e])  or _is_whitespace(rel_cut_words[idx][i][-1]))
            pos_label[b+1:e+1]=pos2id_BIO['I-'+rel_cut_tags[idx][i]]
            pos_label[b]=pos2id_BIO['B-'+rel_cut_tags[idx][i]]
        rel_data_postag.append(pos_label)
    pk.dump(rel_data_postag,open("./middle_data/rel_data_postag.pk","wb"))
else:
    print("loading...")
    rel_data_postag=pk.load(open("./middle_data/rel_data_postag.pk","rb"))
special_major_idx=[2,4,22,32,54]
special_affilate_idx=[[] for i in range(len(id2rels))]
special_affilate_idx[2]=[5]
special_affilate_idx[4]=[0]
special_affilate_idx[22]=[51]
special_affilate_idx[32]=[8]
special_affilate_idx[54]=[6,11,14]
special_pass_idx=[0,5,6,8,11,14,51]
padding_O=[0]*len(new_id2labels)
padding_O[0]=1
padding_category=[0]*len(new_id2labels)
padding_category[5]=1
padding_SEP=[0]*len(new_id2labels)
padding_SEP[6]=1
padding_CLS=[0]*len(new_id2labels)
padding_CLS[7]=1
padding_PAD=[0]*len(new_id2labels)
padding_PAD[8]=1 #for pointer ner

loading dict...
loading...


In [4]:
train_data=[]
with open("./cross_results/train_info.json","r") as r:
    raw_data=r.readlines()
    for d in raw_data:
        train_data.append(json.loads(d))

In [5]:
train_X=[]
train_Y=[]
full_rel_nums=np.zeros((len(train_data)))
rel_index=[]
for i,e in tqdm(enumerate(train_data)):
    cls_logits=np.array(e['cls_logits'])
    triple_num=len(e['spo_list'])
    rels=set()
    entities_num=0
    full_rel_nums[i]=len(text_data[i]['spo_list'])
    for spo in e['spo_list']:
        rels.add(spo['predicate'])
        entities_num+=(1+len(spo['object']))
    rel_num=len(rels)
    for idx in range(len(e['spo_list'])):
        spo=e['spo_list'][idx]
        spo_logits=e['spo_logits'][idx]
        sub_ob_logits=np.concatenate([np.array(spo_logits['subject'])[0],np.array(spo_logits['subject'])[-1],\
                              np.array(spo_logits['object'])[0],np.array(spo_logits['object'])[-1]])
        predicate=spo['predicate'] if spo['predicate'] not in special_rels else spo['predicate']+'_@value'
        sub=spo['subject']
        sub_len=len(sub)
        ob=spo['object']['@value']
        ob_len=len(ob)
        show_time=0
        for plan_spo in e['plan_spo_list']:
            if plan_spo['predicate']==predicate and plan_spo['subject']==sub and plan_spo['object']==ob:
                show_time+=1
                break
        feature=np.concatenate([sub_ob_logits,np.array([show_time,rel_num,entities_num,triple_num]),np.array([sub_len,ob_len])])
        label=0
        if spo in text_data[i]['spo_list']:
            label=1
        train_X.append(feature)
        train_Y.append(label)
        rel_index.append(i)

/home/huangweilin/anaconda3/envs/fjw/lib/python3.6/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [6]:
train_X=np.array(train_X)
train_Y=np.array(train_Y)

In [11]:
full_rel_nums[rel_index[350000]:].sum()

44355.0

In [194]:
p=35423/51495
r=35423/44355
f1=(2*p*r)/(p+r)
p,r,f1

(0.6878920283522673, 0.7986247322737008, 0.7391340636411059)

In [12]:
(((val_pre[350000:]>0.05)==1) & (train_Y[350000:]==1)).sum() #TP

35395

In [13]:
((val_pre[350000:]>0.05)==1).sum(),(train_Y[350000:]==1).sum() #P

(47306, 35423)

In [8]:
data_train = lgb.Dataset(train_X[:],train_Y[:],free_raw_data=False)
data_val=lgb.Dataset(train_X[350000:],train_Y[350000:],free_raw_data=False)
import time
#不带encode 的参数
param = {
    'objective': 'binary', 
    'boost': 'gbdt',
    'metric':'auc',
    #初始参数
    'max_depth': 7,
    'num_leaves': 128,
    'min_data_in_leaf': 100,
    'bagging_fraction': 0.5,
    'bagging_freq': 0,
    'feature_fraction': 1,
    'num_threads': 8,
    'lambda_l1':1e-05,
    'lambda_l2':1e-05,
    'min_split_gain':0,
    'tree_learner': 'serial',
    'verbosity': -1,
    'device': 'cpu',
    'learning_rate': 0.01,
}
start = time.time()
lgb_clf = lgb.train(param, data_train, 4000, valid_sets = [data_train,data_val],\
                    valid_names=["train","val"],verbose_eval=100,early_stopping_rounds=500)# early_stopping_rounds=200
print(time.time() - start)

Training until validation scores don't improve for 500 rounds.
[100]	train's auc: 0.921881	val's auc: 0.922351
[200]	train's auc: 0.926029	val's auc: 0.926458
[300]	train's auc: 0.928586	val's auc: 0.929064
[400]	train's auc: 0.930687	val's auc: 0.931196
[500]	train's auc: 0.932144	val's auc: 0.932665
[600]	train's auc: 0.933352	val's auc: 0.933869
[700]	train's auc: 0.934148	val's auc: 0.934675
[800]	train's auc: 0.934822	val's auc: 0.935353
[900]	train's auc: 0.935484	val's auc: 0.935981
[1000]	train's auc: 0.936137	val's auc: 0.936595
[1100]	train's auc: 0.936767	val's auc: 0.937212
[1200]	train's auc: 0.93735	val's auc: 0.937798
[1300]	train's auc: 0.937859	val's auc: 0.938297
[1400]	train's auc: 0.938372	val's auc: 0.938798
[1500]	train's auc: 0.938831	val's auc: 0.939262
[1600]	train's auc: 0.939305	val's auc: 0.939727
[1700]	train's auc: 0.939794	val's auc: 0.940211
[1800]	train's auc: 0.94032	val's auc: 0.940724
[1900]	train's auc: 0.940779	val's auc: 0.941195
[2000]	train's au

In [15]:
lgb_clf=lgb.Booster(model_file="./model.txt")

In [16]:
val_pre=lgb_clf.predict(train_X[:])
print("acc:",metrics.accuracy_score(train_Y[:],val_pre>0.1))
print("auc:",metrics.roc_auc_score(train_Y[:],val_pre))

acc: 0.8100698638837346
auc: 0.9485282139174224


In [10]:
lgb_clf.save_model('model.txt')